# Modèle from scratch : classification binaire COVID / SAIN (NORMAL)

In [ ]:
# Chargement des modules necessaire :



In [ ]:
# On declare les chemins vers les donnees :


# On declare les dimensions pour les images (224,224) :


# On liste et stocke les chemins des images :


# On affiche le nombre d'image trouve :

In [ ]:
# Visualisation de 6 exemples en 2 lignes et 3 colonnes pour chaque classe :



In [ ]:
# Preprocessing :

# On rescale les images :


# On definit la batch size :


# On prepare les tableaux de donnees depuis les images :



In [ ]:
# On definit l'architecture du model :


# On compile le modeles :


# On affiche le model :



In [ ]:
# On definit les paramétres pour l'entrainement :



In [ ]:
# Entrainement du modele :



In [ ]:
# Visualisation de l'evolution lors de l'entrainement :



In [ ]:
# Basiquement : TRAIN et TEST(1) => Maintenant TRAIN / VALIDATION / TEST(2)

# TRAIN et TEST (1) <=> VALIDATION sont connu du modele // TEST (2 n'est pas connu) externe au training.

# On evalue le modele sur les donnes de validation :


# On evalue le modele sur les donnes de test :


# On test quelque image aleatoirement choisis :

    # Visualisation de l'image puis test puis affichage du resulat

In [ ]:
# On enregistre l'historique et les poids :



In [ ]:
# Data augmentation :

In [ ]:
# Meme processus :

In [ ]:
# Comparaison :

In [ ]:
# Fin du programme ...